In [56]:
#use classicalvenv
import dimod
import json
import numpy as np

In [57]:
def load_ising(file_path):
    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  
    return all_isings_data


def convert_to_dimod_ising(all_isings_data):
    dimod_ising = {}
    for instance in all_isings_data:
        instance_id = instance["instance_id"]
        terms = instance["terms"]
        weights = instance["weights"]

        linear_bias = {}
        quadratic_bias = {}

        for term, weight in zip(terms, weights):
            if len(term) == 1:
                variable = f'x{term[0]}'
                linear_bias[variable] = weight
            elif len(term) == 2:
                var1 = f'x{term[0]}'
                var2 = f'x{term[1]}'
                quadratic_bias[(var1, var2)] = weight

        dimod_ising[instance_id] = (linear_bias, quadratic_bias)
    return dimod_ising

def extract_lowest_energy_solutions(allSolutions):
    results = []
    lowest_energy = allSolutions.first.energy
    for datum in allSolutions.data():
        if datum.energy > lowest_energy:
            break
        sorted_keys = sorted(datum.sample.keys(), key=lambda v: int(v[1:]))
        variable_values = []
        for key in sorted_keys:
            val = datum.sample[key]
            if val > 0:
                variable_values.append(f"+{val}")
            else:
                variable_values.append(str(val))
        
        variable_string = ",".join(variable_values)
        energy = float(datum.energy)
        results.append((variable_string, energy))
            
    return results

In [ ]:
#////////// variables //////////
#isingBatchFile = "batch_Ising_data_Knapsack_6_items_9q_.json"
isingBatchFile = "batch_Ising_data_TSP_9q_.json"
inputFolder = "isingBatches/"

In [59]:
isings = load_ising(inputFolder+isingBatchFile)
dimodIsings = convert_to_dimod_ising(isings)
print(dimodIsings)

{1: ({'x0': -24.0, 'x1': -15.0, 'x2': -21.0, 'x3': -20.0, 'x4': -18.0, 'x5': -20.0, 'x6': -24.0, 'x7': -15.0, 'x8': -21.0}, {('x0', 'x3'): 5.0, ('x1', 'x4'): 5.0, ('x2', 'x5'): 5.0, ('x0', 'x6'): 5.0, ('x1', 'x7'): 5.0, ('x8', 'x2'): 5.0, ('x3', 'x6'): 5.0, ('x4', 'x7'): 5.0, ('x8', 'x5'): 5.0, ('x0', 'x1'): 5.0, ('x0', 'x2'): 5.0, ('x1', 'x2'): 5.0, ('x3', 'x4'): 5.0, ('x3', 'x5'): 5.0, ('x4', 'x5'): 5.0, ('x6', 'x7'): 5.0, ('x8', 'x6'): 5.0, ('x8', 'x7'): 5.0, ('x0', 'x4'): 2.0, ('x3', 'x7'): 2.0, ('x0', 'x5'): 3.0, ('x8', 'x3'): 3.0, ('x1', 'x3'): 2.0, ('x4', 'x6'): 2.0, ('x1', 'x5'): 2.0, ('x8', 'x4'): 2.0, ('x2', 'x3'): 3.0, ('x5', 'x6'): 3.0, ('x2', 'x4'): 2.0, ('x5', 'x7'): 2.0}), 2: ({'x0': -17.5, 'x1': -19.5, 'x2': -17.0, 'x3': -23.0, 'x4': -23.0, 'x5': -18.0, 'x6': -17.5, 'x7': -19.5, 'x8': -17.0}, {('x0', 'x3'): 5.0, ('x1', 'x4'): 5.0, ('x2', 'x5'): 5.0, ('x0', 'x6'): 5.0, ('x1', 'x7'): 5.0, ('x8', 'x2'): 5.0, ('x3', 'x6'): 5.0, ('x4', 'x7'): 5.0, ('x8', 'x5'): 5.0, ('x0', '

In [60]:
solver = dimod.ExactSolver()

globalOptima = {}
for instance, biases in dimodIsings.items():
    #print(instance)
    I = dimodIsings[instance]
    sampleset = solver.sample_ising(I[0], I[1])
    instanceGlobalOptima = extract_lowest_energy_solutions(sampleset)
    globalOptima[instance] = instanceGlobalOptima

print(globalOptima)

{1: [('+1,-1,+1,-1,+1,+1,+1,+1,+1', -90.0), ('+1,+1,+1,-1,+1,+1,+1,-1,+1', -90.0), ('+1,-1,+1,+1,+1,-1,+1,+1,+1', -90.0), ('+1,+1,+1,+1,+1,-1,+1,-1,+1', -90.0)], 2: [('-1,+1,+1,+1,+1,-1,+1,-1,+1', -84.0), ('+1,-1,+1,+1,+1,-1,-1,+1,+1', -84.0)], 3: [('+1,+1,+1,+1,+1,-1,+1,-1,+1', -90.0), ('+1,-1,+1,+1,+1,-1,+1,+1,+1', -90.0)], 4: [('+1,-1,+1,+1,+1,-1,-1,+1,+1', -94.0), ('-1,+1,+1,+1,+1,-1,+1,-1,+1', -94.0)], 5: [('+1,+1,-1,+1,-1,+1,-1,+1,+1', -88.0), ('-1,+1,+1,+1,-1,+1,+1,+1,-1', -88.0)], 6: [('-1,+1,+1,+1,+1,-1,+1,-1,+1', -84.0), ('+1,-1,+1,+1,+1,-1,-1,+1,+1', -84.0)], 7: [('-1,+1,+1,+1,+1,+1,+1,+1,-1', -106.0), ('+1,+1,-1,+1,+1,+1,-1,+1,+1', -106.0)], 8: [('+1,+1,-1,+1,+1,+1,-1,+1,+1', -88.0), ('-1,+1,+1,+1,-1,+1,+1,+1,-1', -88.0), ('-1,+1,+1,+1,+1,+1,+1,+1,-1', -88.0), ('+1,+1,-1,+1,-1,+1,-1,+1,+1', -88.0)], 9: [('+1,+1,+1,+1,-1,-1,+1,+1,+1', -100.0), ('-1,+1,+1,+1,+1,+1,+1,+1,-1', -100.0), ('+1,+1,-1,+1,+1,+1,-1,+1,+1', -100.0), ('+1,+1,+1,-1,+1,-1,+1,+1,+1', -100.0), ('+1,+1,+1,+1

In [61]:
def save_solutions_to_json(solutions_dict, output_filepath):
    output_list = []
    for instance_id, solutions_list in solutions_dict.items():            
        solutions = [sol[0] for sol in solutions_list]
        
        # The cost is the same for all solutions of an instance
        cost = solutions_list[0][1]

        formatted_entry = {
            "instance_id": instance_id,
            "solutions": solutions, # Stored as a list of strings
            "cost": float(cost)
        }
        
        output_list.append(formatted_entry)
        
    with open(output_filepath, 'w') as f:
        json.dump(output_list, f, indent=2)

output_file = inputFolder+'solved_'+isingBatchFile
save_solutions_to_json(globalOptima, output_file)